# Modelo de Redes Neuronales

El modelo de redes neuronales es construido a partir de un conjunto de datos con 4 clases: 

    * wofs
    * bosque
    * wofs_bosque
    * ninguno

In [31]:
%matplotlib inline
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

import keras
import matplotlib.pyplot as plt
from rasterio.plot import show
import xarray as xr

import gdal
import rasterio
import glob
import os
import numpy as np
import pandas as pd
import pandas_profiling as pp

## Lectura del Dataset

In [2]:
data = pd.read_csv('./satellite_dataset-sin-nubes.csv')
data.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,272.0,172.0,74.0,61.0,81.0,62.0,1,0,0,0
1,172.0,158.0,71.0,62.0,107.0,92.0,1,0,0,0
2,172.0,104.0,14.0,1.0,44.0,39.0,1,0,0,0
3,255.0,202.0,103.0,87.0,128.0,104.0,1,0,0,0
4,255.0,197.0,100.0,86.0,113.0,92.0,1,0,0,0


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
blue           1000000 non-null float64
green          1000000 non-null float64
red            1000000 non-null float64
nir            1000000 non-null float64
swir1          1000000 non-null float64
swir2          1000000 non-null float64
wofs           1000000 non-null int64
bosque         1000000 non-null int64
wofs_bosque    1000000 non-null int64
ninguno        1000000 non-null int64
dtypes: float64(6), int64(4)
memory usage: 76.3 MB


In [3]:
# pp.ProfileReport(data)

In [4]:
dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','ninguno'], axis=1)
dataset_X.head()

,blue,green,red,nir,swir1,swir2
0,272.0,172.0,74.0,61.0,81.0,62.0
1,172.0,158.0,71.0,62.0,107.0,92.0
2,172.0,104.0,14.0,1.0,44.0,39.0
3,255.0,202.0,103.0,87.0,128.0,104.0
4,255.0,197.0,100.0,86.0,113.0,92.0


In [5]:
dataset_Y = data[['wofs','bosque','wofs_bosque','ninguno']]
dataset_Y.head()

,wofs,bosque,wofs_bosque,ninguno
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2, random_state=0)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [24]:
# DEFINICIÓN DEL MODELO
# Hay dos maneras de construir modelos en Keras: Secuencial y Funcional
# El modelo secuencial permite construir modelos capa por capa.
# El modelo funcional permite construir modelos mas complicados.

# La capa Flaten transforma los datos de un arreglo bidimensional de 28x28 a un arreglo 
# unidimensional de 784 posiciones(28x28) esto solo formatea el conjutno de datos

# La capa Dense significa que cada neurona en una capa esta conectada a todas las neuronas 
# localizadas en la capa anterior y con todas en la siguiente capa.

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(9,input_dim=6, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

# CONFIGURACIÓN DE PARÁMETROS
# La compilación del modelo comprende la configuración de parámetros 
# usados durante el entrenamiento: algori tmo de optimización, medida 
# de exactitud, etc.

# optimizer: Adam es un algoitmo de optimización. Además es un método de tasa de aprendizaje adaptativo, 
# loss: calcula la diferencia entre la salida y la variable objetivo. Mide la precisión del modelo durante 
# el entrenamiento y queremos minimizar esta función.
# metrics: Son las métricas que se desan calcular durante el entrenamiento. mide la fracción de imágenes que 
# están clasificadas correctamente.

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen de parámetros del modelo
model.summary()

# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=15)

# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 9)                 63        
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 40        
Total params: 103
Trainable params: 103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
800000/800000 [==============================] - 10s 12us/step - loss: 3.6297 - accuracy: 0.7733
Epoch 2/15
800000/800000 [==============================] - 10s 13us/step - loss: 3.5746 - accuracy: 0.7771
Epoch 3/15
800000/800000 [==============================] - 10s 13us/step - loss: 3.5577 - accuracy: 0.7780
Epoch 4/15
800000/800000 [==============================] - 10s 13us/step - loss: 3.5050 - accuracy: 0.7792
Epoch 5/15
800000/800000 [==============================] - 10s 12us/step - loss: 3.26

In [32]:
model.save('model_nn_1.h5')

In [26]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred,axis=1)
y_pred_classes

array([0, 0, 1, ..., 0, 0, 1])

In [27]:
y_test_classes = np.argmax(y_test,axis=1)
y_test_classes

array([0, 0, 1, ..., 0, 0, 1])

## Matrix de Confusión

* Clase 0 (wofs): Agua
* Clase 1 (bosque): Bosque
* Clase 2 (wofs_bosque): Agua o Bosque
* Clase 3 (ninguno): Ninguna de las coverturas anteriores. Puedes ser repreentar una urbanización.

In [28]:
confusion_matrix(y_test_classes, y_pred_classes)

array([[150647,      6,      0,    184],
       [   117,  39275,      0,    820],
       [   376,      2,      0,     28],
       [  1100,   1484,      0,   5961]])

## Precisión y Recall

* Clase 0 (wofs): Agua
* Clase 1 (bosque): Bosque
* Clase 2 (wofs_bosque): Agua o Bosque
* Clase 3 (ninguno): Ninguna de las coverturas anteriores. Puedes ser repreentar una urbanización.

In [29]:
print(classification_report(y_test_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    150837
           1       0.96      0.98      0.97     40212
           2       0.00      0.00      0.00       406
           3       0.85      0.70      0.77      8545

    accuracy                           0.98    200000
   macro avg       0.70      0.67      0.68    200000
weighted avg       0.98      0.98      0.98    200000



# Análisis y Conclusiones

La Clase 2 (agua_bosque) presenta poca representación en el conjunto de datos en general. En particular, esta clase representa el 0.203% del del conjunto de test. Es por ello que el modelo no logra obtener información relevante para ésta clase en particular. Esto se refleja en la baja medida de precisión y recall, es decir, el modelo no logra identificar algúna instancia para esta clase.

Así mismo, la Clase (ninguno) que representa el 4.2725% del conjunto de test. A pesar de la poca representación, se logra tener una precisión y recall de 85% y 70% respectivamente. Probablemente, aquellas instancias de esta clase seleccionadas para el entrenamiento son bastante representativas.

Las clases 0 (agua) y 1 (bosque) que son mayoritarias en el número de instancias presentes en el conjunto de datos. Esto se refleja de iguál forma en el conjunto de test con una representación de 75.41% y 20.11% respectivamente. Finalmente, presentan majores medidas de clasificación y recall.

## Conclusiones

* La falta de balance en las clases genera sesgo en el modelo generado, es por ello que se obtienen mejores medidas de precisión y recall para aquéllas clases mayoritarias (clases 0 y 1). 

* Existen instancias significativas para la clase 3 (ninguno) que permiten al modelo percibir información relevante para la generalización, de lo anterior se tiene que a pesar de tener poca representación esta clase, se logran medidas de precisión y recall por encima del 50%.